## Sitting Posture Inference Server

### Load the model

In [61]:
import sys
import os
import torch

# Add the directory to sys.path
model_dir = "../models/2024-11-24_16-34-03"
sys.path.append(model_dir)

# Import the MLP class
# from model import MLP

### Get test data

In [62]:
import pandas as pd
import numpy as np
import torch

# Load the dataset
data = pd.read_csv("../models/2024-11-24_16-34-03/xy_filtered_keypoints_vectors_mediapipe.csv")

# Extract the specific row for inference
row = data.iloc[217]
print("Data Row:", row)

# Separate features and label
features = row.drop("class").values.astype(np.float32)  # Ensure float32 for PyTorch
true_label = row["class"]

# Load the saved scaler's mean and scale
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.mean_ = np.load("../models/2024-11-24_16-34-03/scaler_mean.npy")  # Load the saved mean
scaler.scale_ = np.load("../models/2024-11-24_16-34-03/scaler_scale.npy")  # Load the saved scale

# Normalize the feature vector using the loaded scaler
features_normalized = scaler.transform([features])  # StandardScaler expects a 2D array

# Convert the normalized features to a PyTorch tensor
features_tensor = torch.tensor(features_normalized, dtype=torch.float32)
print("Normalized Features Tensor:", features_tensor)


Data Row: nose_x                  0.41201
nose_y                 0.361206
left_shoulder_x        0.441131
left_shoulder_y        0.569052
right_shoulder_x       0.210063
right_shoulder_y       0.551094
left_hip_x             0.416628
left_hip_y             1.067228
right_hip_x            0.253567
right_hip_y            1.078241
left_knee_x                 0.0
left_knee_y                 0.0
right_knee_x                0.0
right_knee_y                0.0
left_ankle_x                0.0
left_ankle_y                0.0
right_ankle_x               0.0
right_ankle_y               0.0
shoulder_midpoint_x    0.325597
shoulder_midpoint_y    0.560073
class                    proper
Name: 217, dtype: object
Normalized Features Tensor: tensor([[-0.5676,  0.7229, -0.7042,  1.8019, -1.3408,  1.5607, -0.7194,  2.4101,
         -1.2960,  2.4391, -1.1693, -1.2376, -1.1879, -1.2423, -0.9315, -1.0362,
         -0.9275, -0.9811, -1.0687,  1.7107]])


### Recreate the model

In [63]:
import torch.nn as nn

input_size = features_tensor.shape[1]  # Number of features
num_classes = len(data["class"].unique())  # Number of unique classes

# Define the model architecture to match the saved model
class MLP(nn.Module):
	def __init__(self, input_size, num_classes):
		super(MLP, self).__init__()
		self.model = nn.Sequential(
			nn.Linear(input_size, 256),
			nn.BatchNorm1d(256),
			nn.LeakyReLU(negative_slope=0.01),
			nn.Dropout(0.5),
			nn.Linear(256, 128),
			nn.BatchNorm1d(128),
			nn.LeakyReLU(negative_slope=0.01),
			nn.Dropout(0.5),
			nn.Linear(128, 64),
			nn.LeakyReLU(negative_slope=0.01),
			nn.Linear(64, num_classes)
		)

	def forward(self, x):
		return self.model(x)

model = MLP(input_size=input_size, num_classes=num_classes)

# Load the saved model weights
model.load_state_dict(torch.load("../models/2024-11-24_16-34-03/epochs_150_lr_1e-03_wd_1e-03_acc_8298.pth"))
model.eval()  # Set the model to evaluation mode

C:\Users\User\AppData\Local\Temp\ipykernel_55656\546617223.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("../models/2024-11-24_16-34-

MLP(
  (model): Sequential(
    (0): Linear(in_features=20, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): LeakyReLU(negative_slope=0.01)
    (10): Linear(in_features=64, out_features=4, bias=True)
  )
)

### Do Prediction

In [64]:
# Predict
with torch.no_grad():
    output = model(features_tensor)
    _, predicted_class = torch.max(output, 1)

# Print the results
class_labels = ["crossed_legs", "proper", "slouching", "reclining"]  # Update based on your class order
print(f"True label: {true_label}")
print(f"Predicted label: {predicted_class.item()} ({class_labels[predicted_class.item()]})")

if true_label == class_labels[predicted_class.item()]:
    print("Correct prediction!")

True label: proper
Predicted label: 1 (proper)
Correct prediction!
